In [ ]:
!pip install torch torchvision

In [ ]:
#Import Libraries


from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
args={}
kwargs={}
args['batch_size']=1000 // 2, 16, 128
args['test_batch_size']=1000 // 2, 16, 128
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=5000 // args['batch_size']
args['cuda']=False

In [ ]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
class myLeakyReLU(nn.Module):
  def __init__(self):
    super(myLeakyReLU, self).__init__()

  def forward(self, x):
    self.out=x
    self.out[self.out <= 0] *= 0.1
    return self.out

In [ ]:
class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

        self.bn1 = nn.BatchNorm2d(10)
        self.bn2 = nn.BatchNorm2d(20)

        self.gn1 = nn.GroupNorm(5, 10)
        self.gn2 = nn.GroupNorm(10, 20)

        self.myleakyrelu=myLeakyReLU()

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = self.myleakyrelu(F.max_pool2d(self.conv1(x), 2))
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = self.myleakyrelu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)

In [ ]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net()
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309421
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.311146
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.283718
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.272828
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.265955
Train Epoch: 1 [25000/60000 (42%)]	Loss: 2.273636
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.255631
Train Epoch: 1 [35000/60000 (58%)]	Loss: 2.236557
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.233679
Train Epoch: 1 [45000/60000 (75%)]	Loss: 2.213826
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.195688
Train Epoch: 1 [55000/60000 (92%)]	Loss: 2.176088


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 2.1210, Accuracy: 5576/10000 (56%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.191901
Train Epoch: 2 [5000/60000 (8%)]	Loss: 2.166732
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.117065
Train Epoch: 2 [15000/60000 (25%)]	Loss: 2.086368
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.038630
Train Epoch: 2 [25000/60000 (42%)]	Loss: 1.986470
Train Epoch: 2 [30000/60000 (50%)]	Loss: 1.954143
Train Epoch: 2 [35000/60000 (58%)]	Loss: 1.929523
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.859433
Train Epoch: 2 [45000/60000 (75%)]	Loss: 1.817570
Train Epoch: 2 [50000/60000 (83%)]	Loss: 1.720971
Train Epoch: 2 [55000/60000 (92%)]	Loss: 1.676179

Test set: Average loss: 1.3713, Accuracy: 7383/10000 (74%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.659706
Train Epoch: 3 [5000/60000 (8%)]	Loss: 1.617311
Train Epoch: 3 [10000/60000 (17%)]	Loss: 1.536300
Train Epoch: 3 [15000/60000 (25%)]	Loss: 1.513966
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.451110
Train Epoch: 3 [25000/60000 (42%)]	Loss: 1